In [68]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import datetime as dt
from math import cos,sin
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFE
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
pd.set_option('display.max_columns', 100)
import warnings
warnings.filterwarnings('ignore')

from ipywidgets import interact
import plotly.express as px
import pickle

import ipywidgets
import ipywidgets as widgets
from ipywidgets import interact, HBox, VBox, Output, HTML, Dropdown, Button, Layout, Label
from IPython.display import display, clear_output

import requests
import json

import time

import gc
from scipy.stats import norm

from sklearn.preprocessing import LabelBinarizer
import keras


In [95]:
class TemperatureApp:
    
    """
    Framework for an IPython notebook todo list which allows
    additions and deletions to a sorted list of things to do.
    """
    
    def __init__(self):
        "build the widgets and assemble them into a display."
        with open("global_df_la.pkl","rb") as f:
            self.global_df = pickle.load(f)
        with open("global_ridge_model.pkl","rb") as f:
            self.global_ridge_lm = pickle.load(f)
        with open("global_coefs.pkl","rb") as f:
            self.global_coefs = pickle.load(f)
        with open("global_intercepts.pkl","rb") as f:
            self.global_intercepts = pickle.load(f)
        with open("city_monthly_sample.pkl","rb") as f:
            self.city_monthly_df = pickle.load(f).query("Year < 2058")
        
        self.city_monthly_nn_temp1 = keras.models.load_model("./city_monthly_nn_model.save")

        self.example_cities = ['','Dallas', 'Louisville', 'Washington']
        self.example_city_coords = dict()
        self.example_city_coords['Dallas'] = (32.95, -96.70)
        self.example_city_coords['Louisville'] = (37.78, -85.42)
        self.example_city_coords['Washington'] = (39.38, -76.99)
        
        
        
        
        self.global_ridge_lm_features = ['Gas consumption','Coal consumption','Oil consumption','Gas cumsum', 'Coal cumsum','Oil cumsum'] 
        
        self.city_monthly_nn_columns = ['Year','x','y','z','TempMinus1','TempMinus2', \
             'January','February','March','April','May','June','July','August','September','October','November','December']
        self.city_monthly_nn_temp1_columns = ['Year','x','y','z','TempMinus1', \
             'January','February','March','April','May','June','July','August','September','October','November','December']
  
        style="""
            <style>
                /* enlarges the default jupyter cell outputs, can revert by Cell->Current Outputs->Clear */
                .container { width:1020 !important; } 
                
                /* styles for output widgets */
                .o2 {width:400px; border:1px solid #ddd}
                .o3 {width:400px; border:1px solid #ddd}
                .o4 {width:400px; border:1px solid #ddd}
                .o5 {width:800px; }
                .o5 span {color:red !important}
                
                /* custom styles for testing */
                .style_A {background-color:#fafaaa}
                .style_B {background-color:#faaafa}
                .style_C {background-color:#aafafa}
            </style>
        """
        display(HTML(style))
        self.o1 = Output(layout=Layout(width='400px'))
        self.o2 = Output() 
        self.o2.add_class('o2')
        self.o3 = Output()
        self.o3.add_class('o3')
        self.o4 = Output()
        self.o4.add_class('o4')
        self.o5 = Output()
        self.o5.add_class('o5')
        # create a scene for displaying the outputs, 
        # Output1 on the top row, 2,3, and 4 stacked horizontally in the second row
        scene = VBox([self.o1,
                      HBox([self.o2, self.o3, self.o4]),
                      self.o5
                     ])
        display(scene)
    
        with self.o1:
            display(HTML('<h2>Team Cowboys</h2>'))
        with self.o2:
            self.dd_filter = Dropdown(description='Select Scale', options=['', 'Global Average Temperatures', \
                           'Personalized Report'])
            layout = Layout(width='90px',height='30px')
            layout2 = Layout(width='200px',height='30px')
            
            self.lat_w = widgets.BoundedFloatText(value=0,min=-180,max=180,layout=layout)
            self.lon_w = widgets.BoundedFloatText(value=0,min=-180,max=180,layout=layout)
            
            self.city_w = widgets.Dropdown(options=self.example_cities,value='',description='City:',layout=layout2)
            
            self.lat_box = VBox([Label('Latitude'),self.lat_w])
            self.lon_box = VBox([Label('Longitude'),self.lon_w])
            self.city_box = VBox([Label(''),self.city_w])
            self.coords = HBox([self.lat_box,self.lon_box,self.city_box])

            self.btn = Button(description='Run')
            
            self.btn.on_click(self.display_results)
            
            display(self.dd_filter, self.coords,self.btn)

    def global_graph(self, x): 
        
        
        self.mydf = self.global_df.copy().reset_index()
        self.scaler = 1-((1-self.fuel_scaler.value)/5)
        target_column = 'Oil consumption'
        self.mydf.loc[34:70,target_column] = (self.global_intercepts[target_column] + self.mydf['Year'] * (self.global_coefs[target_column])) * self.scaler

        self.mydf.loc[34:70,'LandAndOceanAverageTemperature'] = self.global_ridge_lm.predict(self.mydf.loc[34:70][self.global_ridge_lm_features])
        self.mydf['Oil cumsum'] = self.mydf['Oil consumption'].cumsum()
        with self.o4:
            clear_output()
            
            sns.lineplot(x='Year',y='LandAndOceanAverageTemperature',data=self.mydf.query("Year <= "+str(self.year_range.value[1]) + " and Year >= "+str(self.year_range.value[0]))).set_title('Average Global Yearly Temperature')
            plt.show()
                        
            
                
            
    def display_results(self, x):
            
        filter_by = self.dd_filter.value


        if filter_by=='':
            with self.o3:
                clear_output()
                print('Please select scale.')
            return

        if filter_by == 'Global Average Temperatures':
            df = self.global_df
            lm = self.global_ridge_lm

        else:
            df = self.city_monthly_df
            lm = self.city_monthly_nn_temp1
        
        if filter_by =='Personalized Report':
        
            with self.o3:
                clear_output()
                
                
                if self.city_w.value != '':
                    lat = self.example_city_coords[self.city_w.value][0]
                    lon = self.example_city_coords[self.city_w.value][1]
                else:
                    lat = self.lat_w.value
                    lon = self.lon_w.value
                    
                api_key = "d7f6fcc66c3bd5ab43f7e2b871cf089a"
                url = "https://api.openweathermap.org/data/2.5/weather?lat="+str(lat)+"&lon="+str(lon)+"&appid=" + str(api_key) + "&units=metric"

                current_temp = json.loads(requests.request("GET",url).text)['main']['temp']
                current_month = dt.datetime.now().strftime('%m').lstrip('0')
                current_month_text = dt.datetime.now().strftime('%B')
                current_year = dt.datetime.now().strftime('%Y')

                start = dt.datetime.now() - dt.timedelta(days=364)
                end = dt.datetime.now() - dt.timedelta(days=363,hours=23)
                start = str(time.mktime(start.timetuple()))[:-2]
                end = str(time.mktime(end.timetuple()))[:-2]
                url = "http://history.openweathermap.org/data/2.5/history/city?lat=" \
                    +str(lat)+"&lon="+str(lon)+"&type=hour&start="+start+ \
                   "&end="+end+"&appid=" \
                    + str(api_key) + "&units=metric"
                
                TempMinus1 = json.loads(requests.request("GET",url).text)['list'][0]['main']['temp']
              
                row = pd.DataFrame()
                row.loc[0,'Year'] = current_year
                row.loc[0,'x'] = cos(lat) * cos(lon)
                row.loc[0,'y'] = cos(lat) * sin(lon)
                row.loc[0,'z'] = sin(lat)
                row.loc[0,'TempMinus1'] = TempMinus1
                row.loc[0,'January']=False
                row.loc[0,'February']=False
                row.loc[0,'March']=False
                row.loc[0,'April']=False
                row.loc[0,'May']=False
                row.loc[0,'June']=False
                row.loc[0,'July']=False
                row.loc[0,'August']=False
                row.loc[0,'September']=False
                row.loc[0,'October']=False
                row.loc[0,'November']=False
                row.loc[0,'December']=False
                row.loc[0,current_month_text]=True

                sc = StandardScaler()

                pred = float(self.city_monthly_nn_temp1.predict(sc.fit_transform(row)))

                mydf = df.query("Latitude == "+str(lat)+" and Longitude =="+str(lon))
                
                if mydf.shape[0] != 0:
                    current_city = mydf.iloc[0]['City']

                lbl = Label(value=f'Latitude: {lat}. Longitude: {lon}. Month: {current_month_text}. Temp: {current_temp} Celsius.')
                lbl.add_class(f'style_{filter_by}')

                display(lbl)

                x = np.arange(pred-15, pred+15, 0.1)
                plt.plot(x, norm.pdf(x, pred, 1.7))
                plt.axvline(x=current_temp, color='red', linestyle='--',label='Temperature This Year')
                plt.axvline(x=TempMinus1, color='blue', linestyle='--',label='Temperature Last Year')
                plt.legend()
                plt.show()

                if current_temp - pred > 0:
                    direction = "higher"
                else:
                    direction = "lower"

                stds = round(abs(current_temp - pred)/1.55,2)


                lbl = Label(value=f'The current temperature is {stds} standard deviations {direction}')
                lbl.add_class(f'style_{filter_by}')
                display(lbl)
                lbl = Label(value=f'from the expected average monthly temperature for {lat},{lon}.')
                lbl.add_class(f'style_{filter_by}')
                display(lbl)
            with self.o4:
                clear_output()
                sns.lineplot(x='Year',y='AverageTemperature',data=df.query('Year < 2040')).set_title('Average Yearly Temperature Projection')
                plt.show()
                months = ['January','February','March','April','May','June','July','August','September','October','November','December']
                
                if mydf.shape[0] != 0:
                    stdDev=1.55
                    month_df = mydf.query("Year ==" +str(current_year))
                    y_list = np.array(month_df['AverageTemperature'])
                    sns.lineplot(x=months,y='AverageTemperature',data=month_df).set_title('Predicted Monthly Temperature for '+ current_city +' in '+str(current_year))
                    plt.fill_between(months, (y_list-stdDev*3), (y_list+stdDev*3), color='grey', alpha=0.3,label='3 StDev')
                    plt.fill_between(months, (y_list-stdDev*2), (y_list+stdDev*2), color='grey', alpha=0.5,label='2 StDev')
                    plt.fill_between(months, (y_list-stdDev), (y_list+stdDev), color='grey', alpha=0.9,label='1 StDev')
                    plt.xticks(rotation=45)
                    plt.grid()
                    plt.legend()
                    plt.ylabel("Degrees (C)")
                    plt.show()

        else:
            with self.o3:
                clear_output()
                
                
                self.year_range = widgets.SelectionRangeSlider(
                    options=list(range(1980,2051)),
                    description='Months to show',
                    disabled=False
                )
                self.fuel_scaler = widgets.FloatSlider(
                                value=1,
                                min=.96,
                                max=1,
                                step=0.0001,
                                description='Oil Scaler:',
                                disabled=False,
                                continuous_update=False,
                                orientation='horizontal',
                                readout=True,
                                readout_format='.2f',
                            )
                self.graph_btn = Button(description='Run')
                self.graph_scene = VBox([self.year_range,
                      self.fuel_scaler,self.graph_btn])
                self.graph_btn.on_click(self.global_graph)
                display(self.graph_scene)
                
            
               
                




            

        with self.o5:
            clear_output()
            display(Label(value= f'Code last run {dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}' ))


d = TemperatureApp()
            
   

    

HTML(value='\n            <style>\n                /* enlarges the default jupyter cell outputs, can revert by…